In [1]:
!pip install ipywidgets tqdm matplotlib numpy --quiet

You should consider upgrading via the 'C:\Users\vries\PycharmProjects\advent_of_code_2021\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [18]:
from pathlib import Path
from typing import List

import numpy as np
from tqdm.notebook import tqdm

import time 

import ipywidgets
import matplotlib.pyplot as plt

from heapq import heappush, heappop
from functools import lru_cache

In [19]:
input_text = Path("../input_data/15").read_text()

In [32]:
risk_map = np.array([[int(x) for x in line]for line in input_text.split()], dtype="int8")
display(risk_map)
display(risk_map.shape)

array([[1, 6, 3, ..., 3, 1, 3],
       [9, 1, 8, ..., 4, 7, 3],
       [4, 2, 1, ..., 4, 5, 4],
       ...,
       [4, 2, 8, ..., 7, 1, 2],
       [9, 2, 5, ..., 9, 1, 6],
       [4, 5, 1, ..., 9, 9, 1]], dtype=int8)

(100, 100)

In [38]:
n_times_bigger = 5  # 1 for part 1, 5 for part 2, more for fun :)

risk_map_complete = np.empty((risk_map.shape[0] * n_times_bigger, risk_map.shape[1] * n_times_bigger), dtype="int32")
for x in range(risk_map_complete.shape[0]):
    for y in range(risk_map_complete.shape[1]):
        mx = int(x / risk_map.shape[0])
        rx = x % risk_map.shape[0]
        my = int(y / risk_map.shape[1])
        ry = y % risk_map.shape[1]
        

        weight = (risk_map[rx, ry])
        for _ in range(mx + my):
            weight += 1
            if weight == 10:
                weight = 1        
        risk_map_complete[x, y] = weight
        
risk_map_complete[0, 0] = 0
display(risk_map_complete)
display(risk_map_complete.shape)

array([[0, 6, 3, ..., 7, 5, 7],
       [9, 1, 8, ..., 8, 2, 7],
       [4, 2, 1, ..., 8, 9, 8],
       ...,
       [8, 6, 3, ..., 6, 9, 1],
       [4, 6, 9, ..., 8, 9, 5],
       [8, 9, 5, ..., 8, 8, 9]])

(500, 500)

In [39]:
class DijkstraNode:
    def __init__(self, x, y, weight):
        self.x = x
        self.y = y
        self.weight = weight
        self.parent = None
        self.distance = np.inf

    @property
    def coordinates(self):
        return self.x, self.y


    def __repr__(self):
        return f"<Dijkstra* node: {self.coordinates} weight {self.weight} distance {self.distance}>"

    
    def __lt__(self, other):
        sum_self =  self.x + self.y
        sum_other = other.x + other.y
        if sum_self != sum_other:
            return sum_self < sum_other
        elif self.x != other.x:
            return self.x < other.x
        elif self.y != other.y:
            return self.y < other.y
        

In [40]:
def visualize_nodes(default_output, current, while_time=None):
    h_map = np.zeros(risk_map_complete.shape, dtype="int32")
    
    vis_time_start = time.perf_counter()
    
    for node in visited:
        h_map[node.coordinates] = node.distance       
    
    default_output.clear_output(wait=True)
    
    with default_output:
        fig = plt.figure(figsize=(20, 20))
        ax = fig.add_subplot(111)
        cax = ax.matshow(h_map)
        fig.colorbar(cax)

        plt.show(fig)
      
        vis_time = time.perf_counter() - vis_time_start
        if while_times:
            while_time = sum(while_times) - max(while_times)
            print(f"While time = {while_time:.3f}s")
        print(f"vis time = {vis_time:.3f}s")
        print(current.distance)


    

In [41]:
all_nodes = []
for x in range(risk_map_complete.shape[0]):
    for y in range(risk_map_complete.shape[1]):
        all_nodes.append(DijkstraNode(x, y, risk_map_complete[x, y]))
        

coordinates_to_node = {n.coordinates: n for n in all_nodes}
parent_to_child_map = dict()
for x in range(risk_map_complete.shape[0]):
    for y in range(risk_map_complete.shape[1]):
        adjacent_coordinates = [(i, j) for i, j in [
            (x - 1, y),
            (x + 1, y),
            (x, y - 1),
            (x, y + 1),
        ] if 0<=i<risk_map_complete.shape[0] and 0<=j<risk_map_complete.shape[1]]

        children = [coordinates_to_node[ac] for ac in adjacent_coordinates]
        parent_to_child_map[(x, y)] = children

In [42]:

# dijkstras again.. sigh

all_nodes = all_nodes # for completeness

start = all_nodes[0]
start.distance = 0
goal = all_nodes[-1]

visited = set()
open_list = list()

heappush(open_list, (start.distance, start))

out = ipywidgets.Output()
display(out)

previous_time = time.perf_counter()
while_times = []
idx = 0
while open_list:
    current_time = time.perf_counter()
    while_time = current_time - previous_time
    while_times.append(while_time)
    previous_time = current_time
    
    distance, current = heappop(open_list)
    visited.add(current)
        
    if (idx:= idx + 1) % 10000 == 0:
        visualize_nodes(out, current, while_time=while_times)
        while_times = []
        
    
    if current == goal:
        visualize_nodes(out, current, while_times)
        display(goal)
        break  # Found shortest path

    for adjacent in parent_to_child_map[current.coordinates]:
        if adjacent in visited:
            continue
        tentative_distance = current.distance + adjacent.weight
        if tentative_distance < adjacent.distance:
            adjacent.distance = tentative_distance
            heappush(open_list, (adjacent.distance, adjacent))   

Output()

<Dijkstra* node: (499, 499) weight 9 distance 2819>

In [43]:
goal

<Dijkstra* node: (499, 499) weight 9 distance 2819>